# 프로젝트 구조
```bash
├── base_line_codes
│   ├── Dataset.py
│   ├── evaluation.py
│   ├── test.py
│   ├── label_maker.py
│   ├── train.py
│   └── model.py
├── logs
├── parameters
├── project.toml
├── project.lock
├── train_data.csv
└── main.ipynb
``` 
<span style="color: red;">위 구조대로 파일을 구성해주세요</span> <br>
<span style="color: red;">parameters, project.lock, logs, train_data.csv 는 자동으로 생성됩니다</span>

# 필요한 라이브러리 설치 및 버전 맞추기

In [1]:
!pip install ruamel_yaml
!pip install urllib3
!pip install poetry --use-feature=2020-resolver
#!pip install tensorboard

In [2]:
import base_line_codes.train
import base_line_codes.label_maker
import tensorboard
import torch
from torch.utils.tensorboard import SummaryWriter
import gc
import os

# 경로 지정 및 캐시 초기화
#### tensor board log 위치 : /logs

In [3]:
writer = SummaryWriter('logs/')

# 캐시 초기화
gc.collect()
torch.cuda.empty_cache()

# train data labeling
### label
```markdown
* mask
    * mask : 0
    * incorrect : 1
    * normal : 2
* gender
    * female : 0
    * male : 1
* age
    * ~30 : 0
    * 30~60 : 1
    * 60~ : 2
```

In [4]:
base_line_codes.label_maker.label_maker()

데이터 수 : 18900


# train
### parameter and default value
```python
train_model(
    LEARNING_RATE=1e-3,
    EPOCH=5,
    BATCH_SIZE=64,
    PRETRAINED=False,   (True, False)
    LABEL='multi',      (multy, mask, gender, age)
    VAL_SIZE=0.2        (0 ~ 1) validation data 의 비율
)
```

In [5]:
base_line_codes.train.train_model(EPOCH=5)

train setting start
train setting end
train start


/opt/ml/code/base_line_codes/train.py:101: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  gender_label = torch.tensor(gender_label).to(device)
/opt/ml/code/base_line_codes/train.py:102: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  age_label = torch.tensor(age_label).to(device)
/opt/ml/code/base_line_codes/train.py:103: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  mask_label = torch.tensor(mask_label).to(device)
/opt/ml/code/base_line_codes/train.py:116: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.c

Iter [  0/237] | Train Loss 2.7231
gender acc 0.4062 | age acc 0.4375 | mask acc 0.7812 | total acc 0.1719
Iter [ 20/237] | Train Loss 2.1500
gender acc 0.5625 | age acc 0.7500 | mask acc 0.6562 | total acc 0.2500
Iter [ 40/237] | Train Loss 1.9443
gender acc 0.7656 | age acc 0.7812 | mask acc 0.7031 | total acc 0.4219
Iter [ 60/237] | Train Loss 2.0600
gender acc 0.7344 | age acc 0.7969 | mask acc 0.7188 | total acc 0.4062
Iter [ 80/237] | Train Loss 1.6659
gender acc 0.6250 | age acc 0.8125 | mask acc 0.7188 | total acc 0.3750
Iter [100/237] | Train Loss 2.0464
gender acc 0.7031 | age acc 0.7188 | mask acc 0.7344 | total acc 0.3906
Iter [120/237] | Train Loss 1.6912
gender acc 0.7812 | age acc 0.8594 | mask acc 0.7031 | total acc 0.4844
Iter [140/237] | Train Loss 1.9509
gender acc 0.7969 | age acc 0.6875 | mask acc 0.7188 | total acc 0.4219
Iter [160/237] | Train Loss 1.3628
gender acc 0.8594 | age acc 0.8438 | mask acc 0.7969 | total acc 0.5781
Iter [180/237] | Train Loss 1.6264
ge

/opt/ml/code/base_line_codes/train.py:101: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  gender_label = torch.tensor(gender_label).to(device)
/opt/ml/code/base_line_codes/train.py:102: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  age_label = torch.tensor(age_label).to(device)
/opt/ml/code/base_line_codes/train.py:103: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  mask_label = torch.tensor(mask_label).to(device)
/opt/ml/code/base_line_codes/train.py:116: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.c

Iter [  0/237] | Train Loss 1.5721
gender acc 0.7500 | age acc 0.7812 | mask acc 0.8125 | total acc 0.4688
Iter [ 20/237] | Train Loss 1.4944
gender acc 0.8438 | age acc 0.7500 | mask acc 0.8750 | total acc 0.5469
Iter [ 40/237] | Train Loss 1.3495
gender acc 0.8750 | age acc 0.7656 | mask acc 0.8594 | total acc 0.5469
Iter [ 60/237] | Train Loss 1.4561
gender acc 0.7656 | age acc 0.7344 | mask acc 0.8750 | total acc 0.4531
Iter [ 80/237] | Train Loss 1.0604
gender acc 0.8750 | age acc 0.8281 | mask acc 0.9219 | total acc 0.6719
Iter [100/237] | Train Loss 1.0822
gender acc 0.8438 | age acc 0.8906 | mask acc 0.8281 | total acc 0.6094
Iter [120/237] | Train Loss 0.9290
gender acc 0.8750 | age acc 0.8281 | mask acc 0.9531 | total acc 0.7031
Iter [140/237] | Train Loss 0.7655
gender acc 0.9688 | age acc 0.8281 | mask acc 0.9375 | total acc 0.7500
Iter [160/237] | Train Loss 0.8336
gender acc 0.9062 | age acc 0.7969 | mask acc 1.0000 | total acc 0.7500
Iter [180/237] | Train Loss 0.8602
ge

...

Iter [  0/237] | Train Loss 0.4826
gender acc 0.9531 | age acc 0.8438 | mask acc 0.9844 | total acc 0.7812
Iter [ 20/237] | Train Loss 0.7762
gender acc 0.9062 | age acc 0.8750 | mask acc 0.9062 | total acc 0.7344
Iter [ 40/237] | Train Loss 0.8478
gender acc 0.9531 | age acc 0.7656 | mask acc 0.9688 | total acc 0.7188
Iter [ 60/237] | Train Loss 0.5276
gender acc 0.9531 | age acc 0.8594 | mask acc 0.9844 | total acc 0.8125
Iter [ 80/237] | Train Loss 0.5298
gender acc 0.9375 | age acc 0.8594 | mask acc 0.9688 | total acc 0.7812
Iter [100/237] | Train Loss 0.7692
gender acc 0.9219 | age acc 0.8281 | mask acc 0.9688 | total acc 0.7500
Iter [120/237] | Train Loss 0.5581
gender acc 0.8906 | age acc 0.8594 | mask acc 1.0000 | total acc 0.7812
Iter [140/237] | Train Loss 0.7834
gender acc 0.9375 | age acc 0.7812 | mask acc 0.9531 | total acc 0.6719
Iter [160/237] | Train Loss 0.4546
gender acc 0.9219 | age acc 0.9219 | mask acc 1.0000 | total acc 0.8594
Iter [180/237] | Train Loss 0.4725
ge

/opt/ml/code/base_line_codes/train.py:101: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  gender_label = torch.tensor(gender_label).to(device)
/opt/ml/code/base_line_codes/train.py:102: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  age_label = torch.tensor(age_label).to(device)
/opt/ml/code/base_line_codes/train.py:103: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  mask_label = torch.tensor(mask_label).to(device)
/opt/ml/code/base_line_codes/train.py:116: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.c

Iter [  0/237] | Train Loss 0.4846
gender acc 0.8438 | age acc 0.9375 | mask acc 1.0000 | total acc 0.8125
Iter [ 20/237] | Train Loss 0.3979
gender acc 0.9531 | age acc 0.8906 | mask acc 0.9844 | total acc 0.8281
Iter [ 40/237] | Train Loss 0.3009
gender acc 0.9844 | age acc 0.9219 | mask acc 1.0000 | total acc 0.9062
Iter [ 60/237] | Train Loss 0.7892
gender acc 0.9062 | age acc 0.8906 | mask acc 0.9688 | total acc 0.8125
Iter [ 80/237] | Train Loss 0.4768
gender acc 0.9375 | age acc 0.8438 | mask acc 0.9844 | total acc 0.7812
Iter [100/237] | Train Loss 0.4968
gender acc 1.0000 | age acc 0.8125 | mask acc 1.0000 | total acc 0.8125
Iter [120/237] | Train Loss 0.6212
gender acc 0.8750 | age acc 0.8438 | mask acc 1.0000 | total acc 0.7344
Iter [140/237] | Train Loss 0.4970
gender acc 0.9531 | age acc 0.8438 | mask acc 0.9844 | total acc 0.7812
Iter [160/237] | Train Loss 0.5423
gender acc 0.9375 | age acc 0.9062 | mask acc 0.9688 | total acc 0.8281
Iter [180/237] | Train Loss 0.5733
ge

/opt/ml/code/base_line_codes/train.py:101: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  gender_label = torch.tensor(gender_label).to(device)
/opt/ml/code/base_line_codes/train.py:102: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  age_label = torch.tensor(age_label).to(device)
/opt/ml/code/base_line_codes/train.py:103: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  mask_label = torch.tensor(mask_label).to(device)
/opt/ml/code/base_line_codes/train.py:116: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.c

Iter [  0/237] | Train Loss 0.1934
gender acc 1.0000 | age acc 0.9688 | mask acc 1.0000 | total acc 0.9688
Iter [ 20/237] | Train Loss 0.3223
gender acc 0.9219 | age acc 0.9531 | mask acc 1.0000 | total acc 0.8906
Iter [ 40/237] | Train Loss 0.3125
gender acc 1.0000 | age acc 0.9062 | mask acc 1.0000 | total acc 0.9062
Iter [ 60/237] | Train Loss 0.2647
gender acc 0.9844 | age acc 0.9062 | mask acc 1.0000 | total acc 0.8906
Iter [ 80/237] | Train Loss 0.4759
gender acc 0.9688 | age acc 0.8750 | mask acc 1.0000 | total acc 0.8438
Iter [100/237] | Train Loss 0.5612
gender acc 0.9062 | age acc 0.8750 | mask acc 1.0000 | total acc 0.7969
Iter [120/237] | Train Loss 0.2539
gender acc 0.9688 | age acc 0.9219 | mask acc 1.0000 | total acc 0.8906
Iter [140/237] | Train Loss 0.4425
gender acc 0.9688 | age acc 0.8906 | mask acc 0.9844 | total acc 0.8594
Iter [160/237] | Train Loss 0.3160
gender acc 0.9844 | age acc 0.9375 | mask acc 1.0000 | total acc 0.9219
Iter [180/237] | Train Loss 0.5380
ge

In [6]:
%load_ext tensorboard
# if not 령태:
%tensorboard --logdir=logs --host=0.0.0.0 --port=30001
# else:
#%tensorboard --logdir=./logs --host=0.0.0.0 --port=30005